In [1]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
from scipy import stats
import scipy

In [2]:
#Define your path to directory with raw particle data
path = "" 

In [ ]:
# for loop that combines all .xls files within path directory. Creates new variable "all_data" as .csv 
# and saves within path directory as "total_particles"
all_data = pd.DataFrame()
for f in glob.glob(path+'\*.xls'):
    df = pd.read_csv(f,sep='\s+')
    all_data = all_data.append(df,ignore_index=True) 
    
all_data.to_csv(path+'/total_particles.csv')

In [ ]:
# Read new total_particle.csv and define variable "area" 
file= path+'/total_particles.csv'
df = pd.read_csv(file,header=0)
area=df['Area']

# transform surface area to equivalent spherical diameter (ESD)
ESD = (2*(area/np.pi)**.5) 

In [ ]:
# define bins (how many, width, and center). Code in this cell modified from Colleen Durkin. 

bins = []
for x in np.arange(1.5,10,.5):
    bin = 2**x
    bins.append(bin)

bin_mids = []
for y in np.arange(0,len(bins)-1):
    mid = bins[y] + (bins[y+1]-bins[y])/2
    bin_mids.append(mid)

bin_width = []
for z in np.arange(0,len(bins)-1):
    width = (bins[z+1]-bins[z])
    bin_width.append(width)

In [ ]:
# Use numpy histgram to sort particle counts into size class bins. This sorted data is then normalized by bin width. 

histogram=np.histogram(ESD,bins=bins,range=None, normed=False, weights=None, density=None) 
n=(histogram[0]/bin_width)*10 #normalized added *10 so no negatives after log transform this will be corrected when count/ml

In [ ]:
# get rid of bins with no data

bin_mids_array=np.asarray(bin_mids) # change bin_mids from tuple to array so can be indexed
ii=np.where(n>0) # index to remove zero data
bin_mids_nonzero=bin_mids_array[ii] # only bins with data
n_nozero=n[ii] # no zero values

In [ ]:
# PSD function will be used in max.min to calculate differences in estimated power equation from real data

def PSD (var, bin_mid, normalized): #var is in format of [#,#], normilized = my n
    normalized_n = normalized#[normalized>0]
    bin_center = bin_mids_nonzero # figure this line out
    Y = (10**var[0])*bin_mids_nonzero**var[1]
    difference = np.sum((np.log(Y)- np.log(n_nozero))**2)
    return difference 

In [ ]:
# optimize the power equation fit using PSD function. Output is variables A and B in equation y = A(x)^B

scipy.optimize.minimize(PSD, [1, -3], args= (bin_mids, n)) # then use these to plot y= over hist with estimates of [1,-3]

In [ ]:
plt.figure()
plt.scatter(bin_mids, n)
plt.plot((10**3.77)*bin_mids_nonzero**(-2.18)) #this is where you add in variable x from optimize.min result 
plt.xlabel('Equivalent Spherical Diameter (ESD) $um$', fontsize=10 )
plt.ylabel('# of particles', fontsize=10)
plt.title('Particle Size Distribution (PSD)', fontsize=20)
#plt.text(300,40, 'n='+total_count,verticalalignment='bottom', horizontalalignment='left',color='black', fontsize=10)
plt.yscale('log')
plt.xscale('log')
plt.axis([1,1000,.1,1000])
plt.show()